# Feature engineering and ETL
In the initial data exploration it was established that the data is skewed , has some outliers and missing values for numerical columns. There are categorical features that need to be one hot encoded. Since multiple models will be used multiple saves will be created depending on the transformations. For eg, Tree based models do not require that data be scaled while it is a good idea to scale data for neural networks.

In total 3 different ETL pipelines are created. One with one hot encoded categorical columns, one with scaled columns and one with scaled and transformed. The Yeo jhonson transformation is used on numerical columns to handle skewness. this should work well with Neural Network.

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

In [3]:
df=pd.read_csv("https://raw.githubusercontent.com/gagan3012/online-shoppers-intention-/master/online_shoppers_intention.csv")
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0.0,-1.0,0.0,-1.0,1.0,-1.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0.0,0.0,0.0,0.0,2.0,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0.0,0.0,0.0,0.0,10.0,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [4]:
df['Administrative']=df['Administrative'].clip(lower=0,upper=20)
df['PageValues']=df['PageValues'].clip(lower=0,upper=250)
df['Administrative_Duration']=df['Administrative_Duration'].clip(lower=0,upper=1500)
df['Informational']=df['Informational'].clip(lower=0,upper=10)
df['ProductRelated_Duration']=df['ProductRelated_Duration'].clip(lower=0,upper=20000)
df['ProductRelated']=df['ProductRelated'].clip(lower=0,upper=400)
df['Informational_Duration']=df['Informational_Duration'].clip(lower=0,upper=1500)

In [5]:
cat_dtype = pd.api.types.CategoricalDtype(categories=['Feb', 'Mar', 'May', 'June', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov','Dec'],
                                          ordered=True)#ordered dtype for month
df['Month']=df['Month'].astype(cat_dtype)


In [6]:
df.shape

(12330, 18)

In [7]:
df.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.313901,79.909572,0.500568,33.929274,31.630318,1187.694533,0.022152,0.043003,5.863980,0.061427,2.124006,2.357097,3.147364,4.069586
std,3.299558,165.983561,1.235324,133.387972,43.102423,1769.998361,0.048427,0.048527,18.173049,0.198917,0.911325,1.717277,2.401591,4.025169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,185.000000,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,8.000000,0.000000,0.000000,18.000000,599.766190,0.003119,0.025124,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.500000,0.000000,0.000000,38.000000,1466.479902,0.016684,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,20.000000,1500.000000,10.000000,1500.000000,400.000000,20000.000000,0.200000,0.200000,250.000000,1.000000,8.000000,13.000000,9.000000,20.000000


In [8]:
df.dtypes

Administrative              float64
Administrative_Duration     float64
Informational               float64
Informational_Duration      float64
ProductRelated              float64
ProductRelated_Duration     float64
BounceRates                 float64
ExitRates                   float64
PageValues                  float64
SpecialDay                  float64
Month                      category
OperatingSystems              int64
Browser                       int64
Region                        int64
TrafficType                   int64
VisitorType                  object
Weekend                        bool
Revenue                        bool
dtype: object

In [9]:
cat_cols=['Weekend','VisitorType','Region','Browser','OperatingSystems','TrafficType']
df[cat_cols]=df[cat_cols].astype('category')
df =df.dropna()

In [10]:
df_encoded=pd.get_dummies(df,columns=cat_cols+['Month'],prefix=cat_cols+['Month'])

In [11]:
numeric_cols = df.select_dtypes(include='float64').columns

In [12]:
scaler = MinMaxScaler()
pt=PowerTransformer()
df_transformed = df_encoded.copy()
df_scaled = df_encoded.copy()
df_scaled[numeric_cols]= scaler.fit_transform(df_encoded[numeric_cols])
df_transformed[numeric_cols]= pt.fit_transform(df_encoded[numeric_cols])

In [13]:
df_scaled.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Feb,Month_Mar,Month_May,Month_June,Month_Jul,Month_Aug,Month_Sep,Month_Oct,Month_Nov,Month_Dec
0,0.0,0.0,0.0,0.0,0.0025,0.000000,1.00,1.00,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0050,0.003200,0.00,0.50,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0025,0.000000,1.00,1.00,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0050,0.000133,0.25,0.70,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0250,0.031375,0.10,0.25,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0


In [14]:
df_transformed.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Feb,Month_Mar,Month_May,Month_June,Month_Jul,Month_Aug,Month_Sep,Month_Oct,Month_Nov,Month_Dec
0,-0.991065,-0.997544,-0.52074,-0.491916,-1.923024,-2.094361,2.207184,1.983626,-0.529787,-0.336232,...,1,0,0,0,0,0,0,0,0,0
1,-0.991065,-0.997544,-0.52074,-0.491916,-1.576413,-1.078325,-0.799476,1.571869,-0.529787,-0.336232,...,1,0,0,0,0,0,0,0,0,0
2,-0.991065,-0.997544,-0.52074,-0.491916,-1.923024,-2.094361,2.207184,1.983626,-0.529787,-0.336232,...,1,0,0,0,0,0,0,0,0,0
3,-0.991065,-0.997544,-0.52074,-0.491916,-1.576413,-1.875374,1.711008,1.833617,-0.529787,-0.336232,...,1,0,0,0,0,0,0,0,0,0
4,-0.991065,-0.997544,-0.52074,-0.491916,-0.445603,0.054466,0.758331,0.724766,-0.529787,-0.336232,...,1,0,0,0,0,0,0,0,0,0


In [15]:
df_encoded.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Feb,Month_Mar,Month_May,Month_June,Month_Jul,Month_Aug,Month_Sep,Month_Oct,Month_Nov,Month_Dec
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.00,0.10,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,2.0,2.666667,0.05,0.14,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,10.0,627.500000,0.02,0.05,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0


In [16]:
!pip install lightgbm 

    100% |################################| 1.2MB 2.4MB/s eta 0:00:01
    100% |################################| 7.1MB 1.1MB/s eta 0:00:01##############  | 6.7MB 41.6MB/s eta 0:00:01
    100% |################################| 20.2MB 813kB/s eta 0:00:01
    100% |################################| 26.1MB 861kB/s eta 0:00:01
    100% |################################| 296kB 3.8MB/s eta 0:00:01
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.


In [17]:
!pip install hyperopt

    100% |################################| 1.9MB 4.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/62/20/4d43e141b5bc426ba38274933ef8e76e85c7adea2c321ecf9ebf7421cedf/numpy-1.18.1-cp36-cp36m-manylinux1_x86_64.whl
    100% |################################| 1.7MB 4.0MB/s eta 0:00:01
    100% |################################| 829kB 4.7MB/s eta 0:00:01
    100% |################################| 61kB 3.3MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/dc/29/162476fd44203116e7980cfbd9352eef9db37c49445d1fec35509022f6aa/scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl
  Stored in directory: /home/spark/shared/.cache/pip/wheels/68/f8/29/b53346a112a07d30a5a84d53f19aeadaa1a474897c0423af91
  Stored in directory: /home/spark/shared/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built networkx future
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
Target directory /home/spark/sh

# Boosted tree

In [18]:
import lightgbm as lgb #light gradient boosted tree
from sklearn.model_selection import train_test_split, StratifiedKFold # train and test split
from sklearn.metrics import balanced_accuracy_score,precision_score# metrics
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK# optimization
import numpy as np
import pandas as pd # reading data

In [19]:
label = df_encoded.pop('Revenue').astype('int')
_,_ = df_scaled.pop('Revenue'),df_transformed.pop('Revenue')

In [20]:
skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)
#Hyperparameter search space for gradient boosed trees
lgb_space = {'lr':hp.loguniform('lr',-6.9,-2.3),'num_leaves':hp.quniform('num_leaves',15,255,1),
         'max_depth':hp.choice('max_depth',[-1,9,12]),'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0)}

In [21]:
def lgb_optimizer(params):
    '''A function to optimize  lgb classifier
    :params:=params , dictionary containing the Hyper-parameters for the classifier
    returns true loss and validation loss
    '''
    if 'num_leaves' in params:
        params['num_leaves']=int(params['num_leaves'])
    if 'max_depth' in params:
        params['max_depth']=int(params['max_depth'])
    val_score=[]
    true_scores=[]
    rd=1
    for train_index, test_index in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_index], X_train.iloc[test_index]
        y_tr, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
        clf = lgb.LGBMClassifier(n_estimators=2000,**params)
        clf.fit(X_tr,y_tr,eval_set=(X_val,y_val),early_stopping_rounds =200,eval_metric='logloss',verbose=False)
        y_pred = clf.predict(X_val)
        y_tr_pred=clf.predict(X_tr)
        score=balanced_accuracy_score(y_val,y_pred,)
        true_score=balanced_accuracy_score(y_tr,y_tr_pred)
        val_score.append(score)
        true_scores.append(true_score)
        rd+=1
    mean,std =np.mean(val_score),np.std(val_score)
    true_mean=np.mean(true_scores)
    print("mean: {}, Std: {}".format(mean,std))
    return {'loss':-mean,'status': STATUS_OK,'true_loss':-true_mean}

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df_encoded,label,test_size=0.2,random_state=42)
trials_encoded = Trials()
best_e= fmin(lgb_optimizer,lgb_space, algo=tpe.suggest, max_evals=20, trials=trials_encoded)

mean: 0.7533833164834423, Std: 0.01233636622840149    
mean: 0.7667746809247153, Std: 0.018128491565918788                               
mean: 0.7622321086024422, Std: 0.01837275651719309                                
mean: 0.7782270561345059, Std: 0.004882559532529377                               
mean: 0.7776755523285965, Std: 0.022936696861284755                               
mean: 0.7674734536111462, Std: 0.012627092092442444                               
mean: 0.7504660817317605, Std: 0.02555809360092492                               
mean: 0.7731760190851572, Std: 0.007798419672713328                              
mean: 0.7822265412651539, Std: 0.01014176365071247                                
mean: 0.7508403805034473, Std: 0.011883528904238818                               
mean: 0.7665384971063591, Std: 0.009015786751632962                               
mean: 0.7709503338527848, Std: 0.0048357227164647016                               
mean: 0.7746482455077024, Std: 0.

In [23]:
X_train,X_test,y_train,y_test = train_test_split(df_scaled,label,test_size=0.2,random_state=42)
trials_scaled = Trials()
best_scaled = fmin(lgb_optimizer, lgb_space, algo=tpe.suggest, max_evals=20, trials=trials_scaled)

mean: 0.7515955326335942, Std: 0.011007345137354104   
mean: 0.780523579641666, Std: 0.014548644372795384                               
mean: 0.7630088989918807, Std: 0.008101828965125544                              
mean: 0.7525878808252406, Std: 0.007890697355819083                             
mean: 0.7552142456351701, Std: 0.009573175207130457                              
mean: 0.7841542396116987, Std: 0.019482676988418796                              
mean: 0.7874331450894336, Std: 0.011235470525126696                              
mean: 0.7681535935256429, Std: 0.01816613214557168                               
mean: 0.765720254735361, Std: 0.012577547055744709                               
mean: 0.7659754774725914, Std: 0.017229980607671586                              
mean: 0.7584103783664895, Std: 0.010917435953608267                               
mean: 0.7665562488518538, Std: 0.00904051752934314                                
mean: 0.7808562702214312, Std: 0.005344005

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df_transformed,label,test_size=0.2,random_state=42)
trials_transformed = Trials()
best_transformed = fmin(lgb_optimizer, lgb_space, algo=tpe.suggest, max_evals=20, trials=trials_transformed)

mean: 0.7707042213038442, Std: 0.026421464264551324   
mean: 0.7757554832961135, Std: 0.01240860224763301                               
mean: 0.7494629925156492, Std: 0.006698934135540784                              
mean: 0.7704461117991309, Std: 0.019710282197254004                              
mean: 0.7780565618974185, Std: 0.018163038397543723                              
mean: 0.7719314161531512, Std: 0.011572866678507734                              
mean: 0.777971977110808, Std: 0.01403921560014026                                
mean: 0.7800540987724365, Std: 0.0068986268143904645                             
mean: 0.7742453102525234, Std: 0.012109994980040558                              
mean: 0.7700757370317282, Std: 0.022126581987335133                              
mean: 0.7557586324970039, Std: 0.01891094766576971                                
mean: 0.7777104122328959, Std: 0.010273912614346696                               
mean: 0.7601800293175607, Std: 0.01422565

In [25]:
def model_trainer(Data,label,params,classifier='lgb',test_size=0.2):
    '''A function to train a model. 
    parameters:
    Data :=pandas dataset or numpy array of features
    label:=pandas dataset or numpy array of labels for features 
    params:=python dict parametes for the model
    model:=keras model object or sklearn classifier object
    test_size:=Float (0,1) fraction for test split
    return
    model:=str 'keras','lgb'
    train_score:=float balanced accuracy train score
    test_score:=float balanced accuracy test score
    returns-trained model,train_score,test_score
    '''
    X_train,X_test,y_train,y_test = train_test_split(Data,label,test_size=test_size,random_state=42)
    if classifier=='lgb':
        if 'num_leaves' in params:
            params['num_leaves']=int(params['num_leaves'])
        if 'max_depth' in params:
            #max_depth=[-1,9,12]#max_depth choice list
            params['max_depth']=-1#int(params['max_depth'])#best returns an index corresponding the choice
        clf = lgb.LGBMClassifier(n_estimators=5000,**params)
        clf.fit(X_train,y_train,eval_set=(X_test,y_test),early_stopping_rounds =200,eval_metric='auc',verbose=False)
        train_pred=clf.predict(X_train)
        test_pred=clf.predict(X_test)
    elif classifier=='keras':
        opt=['adam','sgd']
        if 'optimizer' in params:
            params['optimizer'] =opt[params['optimizer']]
        clf = model(feature_size=X_train.shape[-1],**params)
        clf.fit(X_train,y_train,epochs=20,batch_size=128,verbose=0)
        train_pred=np.round(clf.predict(X_train))
        test_pred=np.round(clf.predict(X_test))
    train_score = balanced_accuracy_score(train_pred,y_train)
    test_score= balanced_accuracy_score(test_pred,y_test)
    return clf,train_score,test_score

In [26]:
clf_encoded,lgb_train_encoded,lgb_test_encoded=model_trainer(df_encoded,label,best_e)
clf_scaled,lgb_train_scaled,lgb_test_scaled=model_trainer(df_scaled,label,best_scaled)
clf_transformed,lgb_train_transformed,lgb_test_transformed=model_trainer(df_transformed,label,best_transformed)

In [27]:
print(best_e)
print(best_scaled)
print(best_transformed)
print("| Dataset Type | Train Score | Test Score |")
print("| Encoded      | {:2f}       | {:2f}     |".format(lgb_train_encoded,lgb_test_encoded))
print("| Scaled       | {:2f}       | {:2f}     |".format(lgb_train_scaled,lgb_test_scaled))
print("| Transformed  | {:2f}       | {:2f}     |".format(lgb_train_scaled,lgb_test_scaled))

{'colsample_bytree': 0.5287787193623354, 'lr': 0.0018176893207910196, 'max_depth': -1, 'num_leaves': 18}
{'colsample_bytree': 0.7129385446110139, 'lr': 0.0024492927508158084, 'max_depth': -1, 'num_leaves': 28}
{'colsample_bytree': 0.7848181582024321, 'lr': 0.01598886788184697, 'max_depth': -1, 'num_leaves': 45}
| Dataset Type | Train Score | Test Score |
| Encoded      | 0.894340       | 0.807394     |
| Scaled       | 0.894238       | 0.810159     |
| Transformed  | 0.894238       | 0.810159     |


# Neural network

In [28]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [29]:
skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)

In [30]:
def model(feature_size,lr=0.1,optimizer='adam',hl_size=128):
    ''' function to define keras model
    :feature:=
    '''
    if optimizer=='adam':
        opt =Adam(lr=lr)
    elif optimizer=='sgd':
        opt = SGD(lr=lr)
    hl_size=int(hl_size)
    model =Sequential()
    model.add(Dense(hl_size,activation='relu',input_shape=(feature_size,)))
    model.add(Dropout(0.25))
    model.add(Dense(hl_size,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['acc'])
    return model

In [31]:
def keras_optimizer(params):
    val_score=[]
    true_scores=[]
    rd=1
    for train_index, test_index in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_index], X_train.iloc[test_index]
        y_tr, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
        nn_model = model(X_train.shape[1],**params)
        cb = EarlyStopping(monitor='val_acc',min_delta=0.001,patience=3)
        nn_model.fit(X_tr,y_tr,validation_data=(X_val,y_val),batch_size=128,callbacks=[cb],epochs=50,verbose=0)
        #_,score= nn_model.evaluate(X_val,y_val)
        #_,true_score=nn_model.evaluate(X_tr,y_tr)
        cv_pred = np.round(nn_model.predict(X_val))
        score=balanced_accuracy_score(y_val,cv_pred)
        y_tr_pred = np.round(nn_model.predict(X_tr))
        true_score=balanced_accuracy_score(y_tr,y_tr_pred)
        
        val_score.append(score)
        true_scores.append(true_score)
        rd+=1
    mean,std =np.mean(val_score),np.std(val_score)
    true_mean=np.mean(true_scores)
    print("mean: {}, Std: {}".format(mean,std))
    return {'loss':-mean,'status': STATUS_OK,'true_loss':-true_mean}

In [32]:
keras_space = {'lr':hp.loguniform('lr',-10,-2.3),'optimizer':hp.choice('optimizer',['adam','sgd'])}

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df_encoded,label,test_size=0.2,random_state=42)
best_nn_encoded=fmin(keras_optimizer,keras_space,algo=tpe.suggest,max_evals=10)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


mean: 0.5, Std: 0.0                                   
mean: 0.5127817066419396, Std: 0.024767383090950997                  
mean: 0.49468519238243297, Std: 0.006431249734539248                                
mean: 0.5, Std: 0.0                                                                 
mean: 0.5, Std: 0.0                                                                 
mean: 0.5347291937170943, Std: 0.039781656501561936                                   
mean: 0.6113991449669646, Std: 0.09616708711323922                                    
mean: 0.5, Std: 0.0                                                                  
mean: 0.5007679676282144, Std: 0.006983776792490679                                  
 90%|█████████ | 9/10 [2:22:21<17:02, 1022.58s/trial, best loss: -0.6113991449669646]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df_scaled,label,test_size=0.2,random_state=42)
best_nn_scaled=fmin(keras_optimizer,keras_space,algo=tpe.suggest,max_evals=10)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df_transformed,label,test_size=0.2,random_state=42)
best_nn_transformed=fmin(keras_optimizer,keras_space,algo=tpe.suggest,max_evals=10)